In [1]:
from pwn import *
from base64 import b64decode, b64encode
from Crypto.Util.Padding import pad

BLOCK = 16
def xor(x,y): return bytes([a^b for a,b in zip(x,y)])

goal = pad(b"I am an authenticated admin, please give me the flag", BLOCK)
goal_blocks = [goal[i:i+BLOCK] for i in range(0, len(goal), BLOCK)]
r = remote("34.162.82.42", 5000)
def is_validly_padded(ct):
    r.recvuntil(b"Your choice: ")
    r.sendline(b"2")
    r.sendline(b64encode(ct))
    res = r.recvline()
    if b"Unknown command!" in res:
        return True
    elif b"Error!" in res:
        return False
    else:
        raise Exception("???")
progress = 0
blocks = [bytes([13,37]*8)]
for goal_block in goal_blocks[::-1]:
    plaintext = [0]*16
    for target_n in range(1,17):
        target = [0]*(16-target_n) + [target_n]*target_n
        for x in range(256):
            plaintext[-target_n] = x
            if is_validly_padded(xor(plaintext, target) + blocks[-1]): break
        progress += 1; print(f"{int(100*progress/len(goal))}%")
    blocks.append(xor(plaintext, goal_block))

[x] Opening connection to 34.162.82.42 on port 5000
[x] Opening connection to 34.162.82.42 on port 5000: Trying 34.162.82.42
[+] Opening connection to 34.162.82.42 on port 5000: Done
1%
3%
4%
6%
7%
9%
10%
12%
14%
15%
17%
18%
20%
21%
23%
25%
26%
28%
29%
31%
32%
34%
35%
37%
39%


EOFError: 